In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab-ml").getOrCreate()
spark

In [10]:
spark.stop()

In [11]:
from pyspark import SparkContext
from pyspark import SparkConf
conf = SparkConf().setAppName("read text file in pyspark")
sc = SparkContext(conf=conf)


In [12]:
sc

<SparkContext master=yarn appName=read text file in pyspark>

# Data Extract with a tried Webpage File

In [13]:
df2 = sc.textFile('s3://commoncrawl/crawl-data/CC-MAIN-2020-10/segments/1581875143455.25/wat/CC-MAIN-20200217235417-20200218025417-00559.warc.wat.gz')

## Extracl Target URL from JSON File

In [14]:
df2.take(31)[30]

'WARC-Target-URI: http://0.ssbet138.cn/WWW.XPJ80886.COM/'

In [15]:
from pyspark.sql.types import StructType
import pyspark
from pyspark.sql.types import StructField

from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql import SQLContext
import json
sqlContext = SQLContext(sc)

In [16]:

schema = StructType([StructField("source", StringType(), True), StructField("target", StringType(), True)])
res = sqlContext.createDataFrame([], schema)
res

DataFrame[source: string, target: string]

## Determine Extract Files Paths

In [18]:

paths = []
for i in range(60):
    if i < 10:
        path = path = 's3://commoncrawl/crawl-data/CC-MAIN-2020-10/segments/1581875143455.25/wat/CC-MAIN-20200217235417-20200218025417-0050' + str(i) +'.warc.wat.gz'
    else:
        path = 's3://commoncrawl/crawl-data/CC-MAIN-2020-10/segments/1581875143455.25/wat/CC-MAIN-20200217235417-20200218025417-005' + str(i) +'.warc.wat.gz'
    paths.append(path)

In [19]:
df_t = sc.textFile('s3://commoncrawl/crawl-data/CC-MAIN-2020-10/segments/1581875143455.25/wat/CC-MAIN-20200217235417-20200218025417-00508.warc.wat.gz')

In [25]:
target_url_str = df_t.take(31)[30]
target_url = target_url_str.split(':')[1:]
target = ''
for x in target_url:
    target += x
    target = target.strip()
target

'http//0-network.bepress.com.library.simmons.edu/explore/social-and-behavioral-sciences/political-science/?facet=discipline%3A%22Law%22&facet=download_type%3A%22PDF%22&facet=institution_title%3A%22Andrews+University%22&facet=subject_facet%3A%22United+States%22'

## Extract Source URLs

In [24]:
links_json = df_t.take(49)[48]
json_obj = json.loads(links_json)
links_container = json_obj['Envelope']['Payload-Metadata']['HTTP-Response-Metadata']['HTML-Metadata']['Links']
links_obj = [x['url'] for x in links_container]
links_obj   

['//assets.bepress.com/20200205/img/dcn/DCsunburst.png',
 '/#/social-and-behavioral-sciences/',
 '/',
 '/social-and-behavioral-sciences/',
 '/social-and-behavioral-sciences/political-science/',
 '/explore/social-and-behavioral-sciences/political-science/',
 '?facet=download_type%3A%22PDF%22&facet=institution_title%3A%22Andrews+University%22&facet=subject_facet%3A%22United+States%22',
 '?facet=discipline%3A%22Law%22&facet=institution_title%3A%22Andrews+University%22&facet=subject_facet%3A%22United+States%22',
 '?facet=discipline%3A%22Law%22&facet=download_type%3A%22PDF%22&facet=subject_facet%3A%22United+States%22',
 '?facet=discipline%3A%22Law%22&facet=download_type%3A%22PDF%22&facet=institution_title%3A%22Andrews+University%22',
 'http://digitalcommons.andrews.edu/honors/140',
 'http://digitalcommons.andrews.edu/honors/140',
 'http://0-www.bepress.com.library.simmons.edu/privacy.html',
 '/about/social-and-behavioral-sciences/political-science/',
 '/',
 '/architecture',
 '/arts-and-huma

## Extraction Part 1

In [28]:
for i, path in enumerate(paths):
    df = sc.textFile(path)
    try:
        links_json = df.take(49)[48]
        json_obj = json.loads(links_json)
        links_container = json_obj['Envelope']['Payload-Metadata']['HTTP-Response-Metadata']['HTML-Metadata']['Links']
        links_obj = [x['url'] for x in links_container]
    
        # target 
        target_url_str = df.take(31)[30]
        target_url = target_url_str.split(':')[1:]
        target = ''
        for x in target_url:
            target += x
        target = target.strip()
    
        print(target)
    except:
        print('under current' + str(i) + 'error')

http//00.auto.sohu.com/d/filter/530100_0_7_0_5_2_5_0_0.html
http//0-network.bepress.com.library.simmons.edu/explore/arts-and-humanities/religion/?facet=discipline%3A%22Music%22&facet=publication_facet%3A%22Yale+Journal+of+Music+%26+Religion%22
http//0-network.bepress.com.library.simmons.edu/explore/arts-and-humanities/religion/?facet=institution_title%3A%22Selected+Works%22&facet=discipline%3A%22Esthetics%22
http//0-network.bepress.com.library.simmons.edu/explore/arts-and-humanities/religion/?facet=publication_type%3A%22Selected+Works%22&facet=institution_title%3A%22SelectedWorks%22&facet=discipline%3A%22Social+and+Behavioral+Sciences%22&facet=download_type%3A%22PDF%22&facet=publication_year%3A%222012%22&facet=subject_facet%3A%22Agua%22
http//0-network.bepress.com.library.simmons.edu/explore/arts-and-humanities/?facet=institution_title%3A%22Marshall+University%22&facet=publication_year%3A%221974%22
http//000.qqw55.com/fashion/popinfo/2020/0204/pic53794401327139.shtml
http//0-network.be

In [29]:
df3 = sc.textFile('s3://commoncrawl/crawl-data/CC-MAIN-2020-10/segments/1581875143455.25/wat/CC-MAIN-20200217235417-20200218025417-00499.warc.wat.gz')

In [30]:
links_json2 = df3.take(49)[48]
json_obj2 = json.loads(links_json2)
links_container2 = json_obj2['Envelope']['Payload-Metadata']['HTTP-Response-Metadata']['HTML-Metadata']['Links']

In [31]:
target_url_str = df3.take(31)[30]
target_url = target_url_str.split(':')[1:]
target = ''
for x in target_url:
    target += x
target = target.strip()
target

'http//0-gate.com/category/anime-catagory/anime-currently-covering/noragami/'

In [32]:
from pyspark.sql.types import StructType
import pyspark
from pyspark.sql.types import StructField

from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

In [33]:
sources = [target for _ in range(len(links_obj))]
schema = StructType([StructField("source", StringType(), True), StructField("target", StringType(), True)])

In [34]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

t = zip(sources, links_obj)
dataframe = sqlContext.createDataFrame(t,schema)


dataframe.take(5)


[Row(source='http//0-gate.com/category/anime-catagory/anime-currently-covering/noragami/', target='/moban/ip138/logo.png'),
 Row(source='http//0-gate.com/category/anime-catagory/anime-currently-covering/noragami/', target='http://0.ssbet138.cn/'),
 Row(source='http//0-gate.com/category/anime-catagory/anime-currently-covering/noragami/', target='javascript:;'),
 Row(source='http//0-gate.com/category/anime-catagory/anime-currently-covering/noragami/', target='http://5iqai.ssbet138.cn/'),
 Row(source='http//0-gate.com/category/anime-catagory/anime-currently-covering/noragami/', target='http://zikba.ssbet138.cn/')]

In [35]:
res2 = dataframe

## Extraction Part 2

In [36]:
for i, path in enumerate(paths):
    df = sc.textFile(path)
    try:
        links_json = df.take(49)[48]
        json_obj = json.loads(links_json)
        links_container = json_obj['Envelope']['Payload-Metadata']['HTTP-Response-Metadata']['HTML-Metadata']['Links']
        links_obj = [x['url'] for x in links_container]
    
        # target 
        target_url_str = df.take(31)[30]
        target_url = target_url_str.split(':')[1:]
        target = ''
        for x in target_url:
            target += x
        target = target.strip()
    
        # df
        sources = [target for _ in range(len(links_obj))]
        schema = StructType([StructField("source", StringType(), True), StructField("target", StringType(), True)])
    
        t = zip(sources, links_obj)
        dataframe = sqlContext.createDataFrame(t,schema)
        #dataframe.show(5)
        res2 = res2.unionByName(dataframe)
        print(i)
    except:
        print('under current' + str(i) + 'error')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
under current15error
16
17
18
19
20
under current21error
22
23
24
25
26
27
28
under current29error
30
under current31error
32
33
under current34error
35
36
37
38
39
under current40error
41
42
43
44
under current45error
under current46error
47
48
under current49error
under current50error
51
52
53
54
55
56
57
58
59


In [37]:
res2.select('source').distinct().show()

+--------------------+
|              source|
+--------------------+
|http//0-network.b...|
|http//00.auto.soh...|
|http//0.ssbet138....|
|http//000.qqw55.c...|
|http//0-network.b...|
|http//000407.cn/r...|
|http//0-network.b...|
|http//0-network.b...|
|http//0-network.b...|
|http//0-network.b...|
|http//00.auto.soh...|
|http//0-network.b...|
|http//000019.cn/r...|
|http//0-network.b...|
|http//0-network.b...|
|http//0-network.b...|
|http//0-gate.com/...|
|http//0-network.b...|
|http//0-euro-hand...|
|http//00.auto.soh...|
+--------------------+
only showing top 20 rows



In [39]:
res2.select('source').distinct().count()

51

In [40]:
paths = []
for i in range(100):
    if i < 10:
        path = path = 's3://commoncrawl/crawl-data/CC-MAIN-2020-10/segments/1581875143455.25/wat/CC-MAIN-20200217235417-20200218025417-0040' + str(i) +'.warc.wat.gz'
    else:
        path = 's3://commoncrawl/crawl-data/CC-MAIN-2020-10/segments/1581875143455.25/wat/CC-MAIN-20200217235417-20200218025417-004' + str(i) +'.warc.wat.gz'
    paths.append(path)

In [41]:
for i, path in enumerate(paths):
    df = sc.textFile(path)
    try:
        links_json = df.take(49)[48]
        json_obj = json.loads(links_json)
        links_container = json_obj['Envelope']['Payload-Metadata']['HTTP-Response-Metadata']['HTML-Metadata']['Links']
        links_obj = [x['url'] for x in links_container]
    
        # target 
        target_url_str = df2.take(31)[30]
        target_url = target_url_str.split(':')[1:]
        target = ''
        for x in target_url:
            target += x
        target = target.strip()
    
        # df
        sources = [target for _ in range(len(links_obj))]
        schema = StructType([StructField("source", StringType(), True), StructField("target", StringType(), True)])
    
        t = zip(sources, links_obj)
        dataframe = sqlContext.createDataFrame(t,schema)
        #dataframe.show(5)
        res2 = res2.unionByName(dataframe)
        print(i)
    except:
        print('under current' + str(i) + 'error')

0
1
2
3
under current4error
5
under current6error
7
8
under current9error
10
under current11error
12
13
14
under current15error
16
17
18
under current19error
20
under current21error
22
23
24
under current25error
26
27
28
29
30
31
32
under current33error
34
35
under current36error
37
38
39
40
under current41error
42
43
44
45
under current46error
47
48
under current49error
50
51
52
53
54
55
56
57
58
59
60
61
62
63
under current64error
65
under current66error
67
under current68error
69
70
71
under current72error
under current73error
74
75
76
under current77error
78
79
under current80error
81
under current82error
under current83error
84
under current85error
86
87
88
89
under current90error
91
under current92error
93
94
95
under current96error
97
98
under current99error


In [42]:
res2.select('source').distinct().show()

+--------------------+
|              source|
+--------------------+
|http//0-network.b...|
|http//00.auto.soh...|
|http//0.ssbet138....|
|http//000.qqw55.c...|
|http//0-network.b...|
|http//000407.cn/r...|
|http//0-network.b...|
|http//0-network.b...|
|http//0-network.b...|
|http//0-network.b...|
|http//00.auto.soh...|
|http//0-network.b...|
|http//000019.cn/r...|
|http//0-network.b...|
|http//0-network.b...|
|http//0-network.b...|
|http//0-gate.com/...|
|http//0-network.b...|
|http//0-euro-hand...|
|http//00.auto.soh...|
+--------------------+
only showing top 20 rows



In [44]:
res2.count()

31566

In [43]:
res2.select('source').distinct().count()

51

# Write Scraped Data to S3

In [45]:

res2.write.parquet("s3://aws-logs-780494741284-us-east-1/dataset/project_graph_data1",mode="overwrite")